In [296]:
import pandas as pd

In [297]:
df = pd.read_csv('data/unclean/2018_transactions_original.csv',
                 parse_dates=['ARRIVAL', 'DEPART', 'BOOKING'],
                 dayfirst=True)
df_units = pd.read_csv('data/unclean/room_number_type_mapping.csv',
                       dtype={'room_number': object})

In [298]:
# merge purchases
df = df.drop_duplicates()

# convert units to codes
df['UNIT'] = df['UNIT'].astype(str)
room_mapping_df = df_units.set_index('room_number')
room_mapping_dict = room_mapping_df.to_dict()['room_type']
df['UNIT_num'] = df['UNIT'].copy()
df['UNIT'] = df['UNIT'].replace(room_mapping_dict)
df['UNIT_num'] = df['UNIT_num'].astype(int)

In [299]:
# create group ID's and group sizes
df['group_id'] = df.groupby(['UNIT_num', 'ARRIVAL', 'DEPART']).ngroup()
group_sizes = df.groupby('group_id').size()
df = df.merge(pd.DataFrame(group_sizes), on='group_id', how='outer')
df = df.rename(index=str, columns={0:'grp_size'})
df = df.set_index('group_id')
df = df.drop(['UNIT_num'], axis=1)

In [300]:
# final sorting by booking date and resno
df = df.sort_values(['BOOKING', 'RESNO'])

# rename booking
df = df.rename(index=str, columns={'BOOKING': 'LOOK_DATE'})

In [301]:
df.to_csv('data/2018_clean.csv')